# Get Lineup Trends

In [ ]:
import json
import requests
import pandas as pd
import time
from tqdm import tqdm

export_DIR = "../data/shiny/"
shiny_export_DIR = "C:/Users/pansr/Documents/shinyNBA/data/"

In [ ]:
teams_response = requests.get("https://api.pbpstats.com/get-teams/nba")
teams = teams_response.json()
teams_dict = teams["teams"]
df_teams = pd.DataFrame(teams_dict)
df_teams = df_teams.rename(columns={"text":"team"})
teams_list = df_teams["id"].to_list()

In [ ]:
dfa = []
for year in range(2023,2024):
    season = str(year) + '-' + str(year+1)[-2:]
    for team in tqdm(teams_list):
        url = "https://api.pbpstats.com/get-team-players-for-season?S"
        params = {
            "Season": season, # To get for multiple seasons, separate seasons by comma
            "SeasonType": "Regular Season",
            "TeamId": team,
        }
        response = requests.get(url, params=params)
        response_json = response.json()
        players = response_json["players"]
        df_players1 = pd.DataFrame.from_dict(players,orient="index",columns=["player"]).reset_index()
        df_players1 = df_players1.rename(columns={"index":"id"})
        df_players1["team"] = team
        df_players1["season"] = season
        time.sleep(0.5)
        dfa.append(df_players1)


In [ ]:
df_players = pd.concat(dfa)
df_players = pd.merge(df_players,df_teams,left_on="team", right_on="id")
df_players = df_players.rename(columns={"id_x":"pid","id_y":"tid","team_y":"team"})
df_players = df_players.drop(columns=["team_x"])

In [ ]:
df_players.to_parquet(export_DIR + "lineup_data.parquet")
df_players.to_parquet(shiny_export_DIR + "lineup_data.parquet")

In [ ]:
dff = df_players.query(f'team == "ATL"')

In [ ]:
dff_t = dff[["pid","player"]]
dff_t = dff_t.set_index("pid")
player_dict = dff_t.to_dict('dict')['player']

In [ ]:
url = "https://api.pbpstats.com/get-game-logs/nba"
params = {
    "Season": "2023-24", # To get for multiple seasons, separate seasons by comma
    "SeasonType": "Regular Season",
    "EntityId": "1626156-1627752-1630559-203076-2544",
    "EntityType": "Lineup" # Use LineupOpponent to get opponent stats
}
response = requests.get(url, params=params)
response_json = response.json()
totals = response_json['single_row_table_data']
game_logs = response_json['multi_row_table_data']

In [ ]:
lineup_in = ["203076","2544","1626156","1627752","1630559"]

In [ ]:
s = ""
for ll in lineup_in:
    s += ll
    s += "-"
s[:-1]

In [ ]:
df = pd.DataFrame(game_logs)

In [ ]:
def get_game_logs(lineup):
    url = "https://api.pbpstats.com/get-game-logs/nba"
    params = {
        "Season": "2023-24", # To get for multiple seasons, separate seasons by comma
        "SeasonType": "Regular Season",
        "EntityId": lineup,
        "EntityType": "Lineup" # Use LineupOpponent to get opponent stats
    }
    response = requests.get(url, params=params)
    response_json = response.json()
    totals = response_json['single_row_table_data']
    game_logs = response_json['multi_row_table_data']
    df_gl = pd.DataFrame(game_logs)
    return df_gl,totals


In [ ]:
df,totals = get_game_logs("1627759-1628369-1628401-201950-204001")

In [ ]:
totals['Minutes']/20

In [ ]:
import datetime as dt

In [ ]:
td = dt.timedelta(minutes=16.05)

In [ ]:
td

In [ ]:
pd.to_datetime(td, format="%H:%M:%S")

In [ ]:
len(df)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plot = plt.figure()

In [ ]:
cols_all = list(df.columns)

In [ ]:
cols_all

In [ ]:
cols_all = list(df.columns)
cols_remove = []
remove_str = ['Penalty','SecondChance','Heave','Lost','BallTurnover','TakeFoul','Excluding','PtRebound','Free Throw','Blocked','ShootingFoulsDrawn','Assisted','ReboundPct']
for col in cols_all:
    if any(c in col for c in remove_str):
        cols_remove.append(col)

In [ ]:
len(cols_remove)

In [ ]:
cols = list(set(cols_all) - set(cols_remove))

In [ ]:
basic_a = [
    'PlusMinus',
    'TsPct',
    'EfgPct',
    'Minutes',
    'TotalPoss',
    'ShotQualityAvg',
    'Points',
    'Pace',
    'Rebounds',
    'Assists',
    'Steals',
    'Blocks',
    'FG2A',
    'FG2M',
    'Fg2Pct',
    'FG3A',
    'FG3M',
    'Fg3Pct',
    'Turnovers',
    'Fouls',
    'DefRebounds',
    'OffRebounds',
    'FTA',
    'FtPoints',
]

shooting_a = [
    'AtRimFGA',
    'AtRimFGM',
    'AtRimAccuracy',
    'AtRimFrequency',
    'ShortMidRangeFGA',
    'ShortMidRangeFGM',
    'ShortMidRangeAccuracy',
    'ShortMidRangeFrequency',
    'LongMidRangeFGA',
    'LongMidRangeFGM',
    'LongMidRangeAccuracy',
    'LongMidRangeFrequency',
    'Corner3FGA',
    'Corner3FGM',
    'Corner3Accuracy',
    'Corner3Frequency',
    'Arc3FGA',
    'Arc3FGM',
    'Arc3Accuracy',
    'Arc3Frequency',
]

misc_a = [
    'AssistPoints',
    'AtRimAssists',
    'Corner3Assists',
    'FoulsDrawn',
    'SecondsPerPossDef',
    'SecondsPerPossOff',
]